In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name)             
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))
        
!ls train

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import random
from PIL import Image


from torchvision import transforms, models
from torchvision.transforms import functional
from torch.utils.data import ConcatDataset
from matplotlib.animation import FuncAnimation

!pip install ultralytics

from ultralytics import YOLO
yolo_model = YOLO('/kaggle/input/clean_dirty_yolo_best/pytorch/default/1/clean_dirty_yolo_best.pt')
directory_cleaned = os.fsencode('/kaggle/working/val/cleaned/')
directory_dirty = os.fsencode('/kaggle/working/val/dirty/')
dir_path_cleaned, dir_path_dirty = '/kaggle/working/val/cleaned/', '/kaggle/working/val/dirty/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


['clean_dirty_yolo_best', 'clean_dirty_resnet52_best', 'platesv2']
After zip extraction:
['.virtual_documents', '__MACOSX', 'plates']
['train', 'test', '.DS_Store']


100%|██████████| 21/21 [00:00<00:00, 5018.25it/s]


cleaned  dirty
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 635.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.7/863.7 kB 5.9 MB/s eta 0:00:0000:0100:01


In [21]:
def yolo_predict(yolo_model,list_of_dir,num_of_augment,test_time = False):
    list_of_results = []
    for each_dir in list_of_dir:
        for file in os.listdir(each_dir):
            filename = os.fsdecode(file)
            if (file.endswith(".jpg")):                             
                if test_time:
                    get_augmented_prediction(num_of_augment,
                                             each_dir,
                                             filename,
                                             list_of_results)
                    
                else:
                    res = yolo_model.predict(each_dir+filename)
                    list_of_results.append(res[0].probs.data)
                    
    return list_of_results  

def get_augmented_prediction(num_of_augment,current_dir,filename,list_of_results):
    list_of_images = []
    for i in range(num_of_augment):
        pred_transforms = transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(brightness=(0.5,1.5),
                                       contrast=(1), 
                                       saturation=(0.5,1.5),
                                       hue=(-0.1,0.1))         
                ])

        img = Image.open(current_dir+filename)
        img = pred_transforms(img)
        list_of_images.append(img)
    res = yolo_model.predict(list_of_images)
    list_of_tensors = []
    for each_elem in res:
        list_of_tensors.append(each_elem.probs.data)

    inference_result = torch.stack(list_of_tensors)
    inference_result = torch.mean(inference_result,dim=0)
    list_of_results.append(inference_result)

def create_submission_file(test_img_paths, test_predictions,file_name):
    submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
    submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
    submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
    submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
    submission_df.set_index('id', inplace=True)
    submission_df.to_csv(file_name + ".csv")
    submission_df.head(n=6)
    
           

In [22]:
res = yolo_predict(yolo_model,
                   list_of_dir = [dir_path_cleaned, dir_path_dirty],
                   num_of_augment = 30,
                   test_time = True)


0: 640x640 dirty 0.75, cleaned 0.25, 31.7ms
1: 640x640 dirty 1.00, cleaned 0.00, 31.7ms
2: 640x640 cleaned 0.86, dirty 0.14, 31.7ms
3: 640x640 cleaned 0.51, dirty 0.49, 31.7ms
4: 640x640 cleaned 0.66, dirty 0.34, 31.7ms
5: 640x640 cleaned 0.73, dirty 0.27, 31.7ms
6: 640x640 cleaned 0.89, dirty 0.11, 31.7ms
7: 640x640 cleaned 0.88, dirty 0.12, 31.7ms
8: 640x640 cleaned 0.80, dirty 0.20, 31.7ms
9: 640x640 cleaned 0.85, dirty 0.15, 31.7ms
10: 640x640 dirty 0.71, cleaned 0.29, 31.7ms
11: 640x640 cleaned 0.89, dirty 0.11, 31.7ms
12: 640x640 dirty 0.76, cleaned 0.24, 31.7ms
13: 640x640 dirty 0.58, cleaned 0.42, 31.7ms
14: 640x640 cleaned 0.56, dirty 0.44, 31.7ms
15: 640x640 dirty 0.60, cleaned 0.40, 31.7ms
16: 640x640 cleaned 0.68, dirty 0.32, 31.7ms
17: 640x640 cleaned 0.51, dirty 0.49, 31.7ms
18: 640x640 dirty 0.73, cleaned 0.27, 31.7ms
19: 640x640 cleaned 0.83, dirty 0.17, 31.7ms
20: 640x640 cleaned 0.88, dirty 0.12, 31.7ms
21: 640x640 dirty 0.66, cleaned 0.34, 31.7ms
22: 640x640 cleaned

In [23]:
res

[tensor([0.6078, 0.3922], device='cuda:0'),
 tensor([0.5910, 0.4090], device='cuda:0'),
 tensor([0.9374, 0.0626], device='cuda:0'),
 tensor([0.5303, 0.4697], device='cuda:0'),
 tensor([0.2116, 0.7884], device='cuda:0'),
 tensor([0.1712, 0.8288], device='cuda:0'),
 tensor([0.1144, 0.8856], device='cuda:0'),
 tensor([0.1468, 0.8532], device='cuda:0')]

In [30]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, shuffle=False, num_workers=0)



test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    test_img_paths.extend(paths)



100%|██████████| 744/744 [00:02<00:00, 330.95it/s]


In [31]:
preds = yolo_predict(yolo_model,
                   list_of_dir = ['/kaggle/working/test/unknown/'],
                   num_of_augment = 30,
                   test_time = False)


image 1/1 /kaggle/working/test/unknown/0115.jpg: 640x640 dirty 0.72, cleaned 0.28, 61.6ms
Speed: 14.2ms preprocess, 61.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/test/unknown/0039.jpg: 640x640 dirty 0.81, cleaned 0.19, 61.5ms
Speed: 10.9ms preprocess, 61.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/test/unknown/0007.jpg: 640x640 dirty 0.94, cleaned 0.06, 42.6ms
Speed: 10.7ms preprocess, 42.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/test/unknown/0577.jpg: 640x640 dirty 0.79, cleaned 0.21, 42.5ms
Speed: 10.4ms preprocess, 42.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/test/unknown/0203.jpg: 640x640 dirty 0.80, cleaned 0.20, 42.5ms
Speed: 10.6ms preprocess, 42.5ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/test/unknown/0725.jpg: 640x6

In [32]:
create_submission_file(test_img_paths, preds,'submission_yolo')

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [29]:
create_submission_file(test_img_paths, preds,'submission_yolo')

[tensor([0.2830, 0.7170], device='cuda:0'),
 tensor([0.1946, 0.8054], device='cuda:0'),
 tensor([0.0645, 0.9355], device='cuda:0'),
 tensor([0.2090, 0.7910], device='cuda:0'),
 tensor([0.1985, 0.8015], device='cuda:0'),
 tensor([0.1985, 0.8015], device='cuda:0'),
 tensor([0.2047, 0.7953], device='cuda:0'),
 tensor([0.1846, 0.8154], device='cuda:0'),
 tensor([0.0814, 0.9186], device='cuda:0'),
 tensor([0.1507, 0.8493], device='cuda:0'),
 tensor([0.1335, 0.8665], device='cuda:0'),
 tensor([0.1136, 0.8864], device='cuda:0'),
 tensor([0.2026, 0.7974], device='cuda:0'),
 tensor([0.2777, 0.7223], device='cuda:0'),
 tensor([0.0085, 0.9915], device='cuda:0'),
 tensor([0.0811, 0.9189], device='cuda:0'),
 tensor([0.0264, 0.9736], device='cuda:0'),
 tensor([0.0355, 0.9645], device='cuda:0'),
 tensor([0.3044, 0.6956], device='cuda:0'),
 tensor([0.0424, 0.9576], device='cuda:0'),
 tensor([0.0475, 0.9525], device='cuda:0'),
 tensor([0.3244, 0.6756], device='cuda:0'),
 tensor([0.7270, 0.2730], device

In [47]:
results = yolo_predict(yolo_model,dir_path_cleaned, dir_path_dirty, num_of_augment=30,test_time = True )
loss = torch.nn.CrossEntropyLoss()
target = torch.tensor([[1.,0.],[1.,0.],[1.,0.],[1.,0.],[0.,1.],[0.,1.],[0.,1.],[0.,1.]])
results = torch.stack(results)
loss_val = loss(results.cpu(),target)

results2 = yolo_predict(yolo_model,dir_path_cleaned, dir_path_dirty, num_of_augment=5)
results2 = torch.stack(results2)
target = torch.tensor([[1.,0.],[1.,0.],[1.,0.],[1.,0.],[0.,1.],[0.,1.],[0.,1.],[0.,1.]])
loss2 = torch.nn.CrossEntropyLoss()
loss_val2 = loss(results2.cpu(),target)


0: 640x640 dirty 0.52, cleaned 0.48, 40.2ms
1: 640x640 cleaned 0.85, dirty 0.15, 40.2ms
2: 640x640 cleaned 0.58, dirty 0.42, 40.2ms
3: 640x640 cleaned 0.80, dirty 0.20, 40.2ms
4: 640x640 cleaned 0.81, dirty 0.19, 40.2ms
5: 640x640 cleaned 0.60, dirty 0.40, 40.2ms
6: 640x640 cleaned 0.70, dirty 0.30, 40.2ms
7: 640x640 dirty 0.55, cleaned 0.45, 40.2ms
8: 640x640 cleaned 0.86, dirty 0.14, 40.2ms
9: 640x640 cleaned 0.90, dirty 0.10, 40.2ms
10: 640x640 dirty 0.51, cleaned 0.49, 40.2ms
11: 640x640 cleaned 0.54, dirty 0.46, 40.2ms
12: 640x640 cleaned 0.89, dirty 0.11, 40.2ms
13: 640x640 cleaned 0.64, dirty 0.36, 40.2ms
14: 640x640 cleaned 0.51, dirty 0.49, 40.2ms
15: 640x640 dirty 0.68, cleaned 0.32, 40.2ms
16: 640x640 dirty 0.60, cleaned 0.40, 40.2ms
17: 640x640 cleaned 0.66, dirty 0.34, 40.2ms
18: 640x640 cleaned 0.64, dirty 0.36, 40.2ms
19: 640x640 cleaned 0.84, dirty 0.16, 40.2ms
20: 640x640 dirty 0.67, cleaned 0.33, 40.2ms
21: 640x640 cleaned 0.83, dirty 0.17, 40.2ms
22: 640x640 cleaned

tensor(0.4466)

In [30]:
resnet_model = models.resnet152()
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, 2)
resnet_model.load_state_dict(torch.load('/kaggle/input/clean_dirty_resnet52_best/pytorch/default/1/clean_dirty_resnet52best.pt'))

<All keys matched successfully>

In [32]:
resnet_model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = resnet_model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

create_submission_file(test_img_paths, test_predictions,'submission_resnet')

100%|██████████| 744/744 [02:51<00:00,  4.33it/s]


In [34]:
create_submission_file(test_img_paths, test_predictions,'submission_resnet')

In [36]:
test_predictions.shape

(744,)